# Import libraries

In [1]:
import os
import csv
import time
import pickle
import numpy as np
import tensorflow as tf

from models.deep_crnn_model import *
from models.model_builders.create_dataset import *

# Model parameters

In [2]:
csv_files_train = "../data/iamHandwriting/train.csv"

output_model_dir = "../tf_output/estimator/"
output_preds_dir = "../tf_output/prediction/"
output_graph_dir = "../tf_output/graph/"

n_epochs = 10

batch_size=64

learning_rate=1e-3
learning_decay_rate=0.95
learning_decay_steps=5000

input_shape=(129, 369) #iam shape
# input_shape=(117, 1669) # bentham shape1
# input_shape=(181, 1833) # bentham shape2

optimizer='adam'

# alphabet=" !\"#&'()*+,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXY[]_abcdefghijklmnopqrstuvwxyz|"
alphabet="abcdefghijklmnopqrstuvwxyz"

csv_delimiter='\t'

# Run model

** Create model ops **

In [4]:
input_tensor = tf.placeholder(tf.float32, [None, input_shape[0], input_shape[1], 1])
labels = tf.placeholder(tf.string, [None])

out = deep_crnn(input_tensor, labels, input_shape, alphabet, batch_size)
train_op, loss_ctc, CER, accuracy, predictions_dict = out

** Load dataset **

In [5]:
out = create_iterator(csv_files_train, input_shape, batch_size)
dataset, iterator, next_batch, datasize = out

** Train model **

In [ ]:
saver = tf.train.Saver()

losses = []
cers = []
accs = []

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    sess.run(tf.tables_initializer())
#     saver.restore(sess, output_model_dir+"/model.ckpt")
    
    writer = tf.summary.FileWriter(output_graph_dir, sess.graph)
    n_batches = int(datasize / batch_size)
    for i in range(n_epochs):
        print("---------------------------------------------------------")
        print("Starting epoch", i)
        sess.run(iterator.initializer)
        for j in range(n_batches):
            try:
                _, cer, acc, loss, preddict, scp, pc = sess.run([train_op, CER, accuracy, loss_ctc, predictions_dict],
                             feed_dict={input_tensor: input_tensor_b, labels: labels_b})
                losses.append(loss)
                cers.append(cer)
                accs.append(accs)
                pickle.dump([cers, accs, losses, preddict], open(output_model_dir+"/preds.pkl", "wb"))
                saver.save(sess, output_model_dir+"/model.ckpt")
                
                print('batch: {0}, loss: {3} \n\tCER: {1}, accuracy: {2}'.format(j, cer, acc, loss))
            except:
                print("Error at ", j)
        print('Avg Epoch time: {0} seconds'.format((time.time() - start_time)/(1.0*(i+1))))
            
    print('Total time: {0} seconds'.format(time.time() - start_time))
    print('Optimization Finished!') 

---------------------------------------------------------
Starting epoch 0
batch: 0, loss: 274.429443359375 
	CER: 15.312127113342285, accuracy: [ 0.  0.]
batch: 1, loss: 131.7049560546875 
	CER: 3.5446362495422363, accuracy: [ 0.  0.]
